In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# from sutils import *
import os, json
from glob import glob

import tensorflow as tf

import keras
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout, Flatten,Input, BatchNormalization, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

import multi_gpu

Using TensorFlow backend.


In [2]:
print(tf.__version__)
print(keras.__version__)

1.3.0
2.0.8


In [3]:
fnames =  glob('/../../nfs/p4/shared/datasets/quick_draw/numpy_bitmap/*')
# for entry in fnames:
#     print entry
print 'Number of classes: ', len(fnames) 

Number of classes:  345


In [4]:
## Creating the training data set
Npc = 8000 # Number of images to use for each class 
Npc_t = 2000  # Number of images to use for testing for each class 
n_class = 100 # Number of classes to use 

for i, entry in enumerate(fnames):
    print i
    if i >= n_class:
        break
    data = np.load(entry)[:Npc]
    labels =  np.zeros([data.shape[0],n_class])
    labels[:,i] = 1
    if i == 0:
        X = data[:Npc]
        targets = labels
    else:
        X = np.vstack([X, data])
        targets = np.vstack([targets,labels])
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [5]:
print 'Input shape:', X.shape
print 'Target shape:', targets.shape

Input shape: (800000, 784)
Target shape: (800000, 100)


In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, targets, test_size=0.33, random_state=42)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

In [7]:
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0],28,28,1)

In [8]:
Inp = Input(shape=(28,28,1),name = 'Input_layer')

#ConvBlock 01
conv01 = Conv2D(64, (5, 5), padding='same',activation = 'relu', input_shape=Inp.shape,name = 'Conv01_layer')(Inp)
conv01 = BatchNormalization()(conv01)
conv02 = Conv2D(64, (5, 5),activation = 'relu',name = 'Conv02_layer')(conv01)
maxpool_01 = MaxPooling2D(pool_size=(2, 2),name = 'MaxPool01_layer')(conv02)
drop01 = Dropout(0.25,name = 'Dropout01_layer')(maxpool_01)

#Convblock 02
conv03 = Conv2D(128, (3, 3), padding='same',activation = 'relu',name = 'Conv03_layer')(drop01)
conv03 = BatchNormalization()(conv03)
conv04 = Conv2D(128, (3, 3),activation = 'relu',name = 'Conv04_layer')(conv03)
maxpool_02 = MaxPooling2D(pool_size=(2, 2),name = 'MaxPool02_layer')(conv04)
drop02 = Dropout(0.25,name = 'Dropout02_layer')(maxpool_02)

# Fully Connected Dense block
x = Flatten(name = 'Flatten_layer')(drop02)
x = Dense(512, activation='relu',name = 'Dense01_layer')(x)
x = Dropout(0.5,name = 'Dropout03_layer')(x)
x = BatchNormalization()(x)
logits_layer = Dense(y_train.shape[1], name= 'logits_layer')(x)
output = Activation('softmax',name = 'Sofftmax_layer')(logits_layer)

In [9]:
# Define model inputs and output
model = Model(Inp, output)

In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input_layer (InputLayer)     (None, 28, 28, 1)         0         
_________________________________________________________________
Conv01_layer (Conv2D)        (None, 28, 28, 64)        1664      
_________________________________________________________________
batch_normalization_1 (Batch (None, 28, 28, 64)        256       
_________________________________________________________________
Conv02_layer (Conv2D)        (None, 24, 24, 64)        102464    
_________________________________________________________________
MaxPool01_layer (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
Dropout01_layer (Dropout)    (None, 12, 12, 64)        0         
_________________________________________________________________
Conv03_layer (Conv2D)        (None, 12, 12, 128)       73856     
__________

In [11]:
model = multi_gpu.make_parallel(model,2)

multi_gpu.py:44: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  merged.append(merge(outputs, mode='concat', concat_axis=0))
/nfs/p4/ceusers/limpin/.venv2.7-gpu/local/lib/python2.7/site-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
multi_gpu.py:46: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=[<tf.Tenso..., inputs=[<tf.Tenso...)`
  return Model(input=model.inputs, output=merged)


In [12]:
# initiate RMSprop optimizer

opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [13]:
print('Using real-time data augmentation.')
# This will do preprocessing and realtime data augmentation:
datagen = ImageDataGenerator(
    rotation_range=30,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False)  # randomly flip images

Using real-time data augmentation.


In [14]:
# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(X_train)

In [15]:
batch_size = 128
epochs = 100
data_augmentation = True
# num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'cifar10_simple_CNN.h5'

In [16]:
hist = model.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size),
                            steps_per_epoch=X_train.shape[0] // batch_size,
                            epochs=epochs,
#                             validation_data=(X_test, y_test),
                            workers=50)

Epoch 1/100
4187/4187 [==============================] - 294s - loss: 3.2346 - acc: 0.2403   
Epoch 2/100
4187/4187 [==============================] - 251s - loss: 2.2575 - acc: 0.4328   
Epoch 3/100
4187/4187 [==============================] - 252s - loss: 1.9345 - acc: 0.5065   
Epoch 4/100
4187/4187 [==============================] - 251s - loss: 1.7739 - acc: 0.5438   
Epoch 5/100
4187/4187 [==============================] - 251s - loss: 1.6724 - acc: 0.5694   
Epoch 6/100
4187/4187 [==============================] - 247s - loss: 1.6021 - acc: 0.5869   
Epoch 7/100
4187/4187 [==============================] - 251s - loss: 1.5525 - acc: 0.6001   
Epoch 8/100
4187/4187 [==============================] - 246s - loss: 1.5144 - acc: 0.6086   
Epoch 9/100
4187/4187 [==============================] - 245s - loss: 1.4858 - acc: 0.6165   
Epoch 10/100
4187/4187 [==============================] - 250s - loss: 1.4596 - acc: 0.6233   
Epoch 11/100
4187/4187 [==============================] - 2

KeyboardInterrupt: 

In [ ]:
def plot_train(hist):
    h = hist.history
    if 'acc' in h:
        meas='acc'
        loc='lower right'
    else:
        meas='loss'
        loc='upper right'
    plt.plot(hist.history[meas])
    plt.plot(hist.history['val_'+meas])
    plt.title('model '+meas)
    plt.ylabel(meas)
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc=loc)
    

In [ ]:
plot_train(hist)